In [1]:
import numpy as np
from pycuda import driver, compiler, gpuarray, tools

# -- initialize the device
import pycuda.autoinit

In [2]:
kernel_code_template = """
__global__ void MatrixMulKernel(float *a, float *b, float *c)
{
    // 2D Thread ID (assuming that only *one* block will be executed)
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int x_size = blockDim.x;

    // sum is used to store the element of the matrix
    // that is computed by the thread
    float sum = 0;

    // Each thread loads one row of M and one column of N, 
    //   to produce one element of P.
    for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
        float Aelement = a[ty * %(MATRIX_SIZE)s + k];
        float Belement = b[k * x_size + tx];
        sum += Aelement * Belement;
    }

    // Write the matrix to device memory;
    // each thread writes one element
    c[ty * x_size + tx] = sum;
}
"""

In [3]:
MATRIX_SIZE = 4000
SPLIT_SIZE = 1000

kernel_code = kernel_code_template % {
    'MATRIX_SIZE': MATRIX_SIZE
    }

# compile the kernel code 
mod = compiler.SourceModule(kernel_code)

# get the kernel function from the compiled module
matrixmul = mod.get_function("MatrixMulKernel")

In [4]:
a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
c_cpu = np.empty((MATRIX_SIZE, MATRIX_SIZE))
# create empty gpu array for the result (C = A * B)
c_gpu = gpuarray.empty((SPLIT_SIZE, SPLIT_SIZE), np.float32)

In [5]:
for x in range(0, MATRIX_SIZE, SPLIT_SIZE):
    for y in range(0, MATRIX_SIZE, SPLIT_SIZE):
        # transfer host (CPU) memory to device (GPU) memory
        a_gpu = gpuarray.to_gpu(a_cpu[y:y+SPLIT_SIZE, : ])
        b_gpu = gpuarray.to_gpu(b_cpu[ :, x:x+SPLIT_SIZE])

        matrixmul( a_gpu, b_gpu, c_gpu, block = (SPLIT_SIZE, SPLIT_SIZE, 1) )
        c_cpu[y:y+SPLIT_SIZE, x:x+SPLIT_SIZE] = c_gpu.get()

LogicError: cuFuncSetBlockShape failed: invalid argument

In [6]:
kernel_code_template = """
__global__ void MatrixMulKernel(float *a, float *b, float *c)
{
    // 2D Thread ID (assuming that only *one* block will be executed)
    int tx = blockIdx.x;
    int ty = blockIdx.y;
    int x_size = gridDim.x;

    // sum is used to store the element of the matrix
    // that is computed by the thread
    float sum = 0;

    // Each thread loads one row of M and one column of N, 
    //   to produce one element of P.
    for (int k = 0; k < %(MATRIX_SIZE)s; ++k) {
        float Aelement = a[ty * %(MATRIX_SIZE)s + k];
        float Belement = b[k * x_size + tx];
        sum += Aelement * Belement;
    }

    // Write the matrix to device memory;
    // each thread writes one element
    c[ty * x_size + tx] = sum;
}
"""

In [7]:
MATRIX_SIZE = 4000
SPLIT_SIZE = 1000

kernel_code = kernel_code_template % {
    'MATRIX_SIZE': MATRIX_SIZE
    }

# compile the kernel code 
mod = compiler.SourceModule(kernel_code)

# get the kernel function from the compiled module
matrixmul = mod.get_function("MatrixMulKernel")

In [8]:
for x in range(0, MATRIX_SIZE, SPLIT_SIZE):
    for y in range(0, MATRIX_SIZE, SPLIT_SIZE):
        # transfer host (CPU) memory to device (GPU) memory
        a_gpu = gpuarray.to_gpu(a_cpu[y:y+SPLIT_SIZE, : ])
        b_gpu = gpuarray.to_gpu(b_cpu[ :, x:x+SPLIT_SIZE])

        matrixmul( a_gpu, b_gpu, c_gpu, grid = (SPLIT_SIZE, SPLIT_SIZE), block=(1,1,1))
        c_cpu[y:y+SPLIT_SIZE, x:x+SPLIT_SIZE] = c_gpu.get()